# **Space X  Falcon 9 First Stage Landing Prediction**
## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia
- link
https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


In [1]:
import sys
import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd
import scrapy

In [2]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    

### TASK 1: Request the Falcon9 Launch Wiki page from its URL

In [3]:
url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

In [4]:
tables=pd.read_html(url)
# tables

In [5]:
data=requests.get(url).text

In [6]:
soup=BeautifulSoup(data,'html.parser')

In [7]:
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

In [8]:
html_tables=soup.find_all('table')

In [9]:
first_launch_table=html_tables[2]
# first_launch_table

In [10]:
columns=[]
elements= soup.find_all('th')
for row in range(len(elements)):
    try:
        name=extract_column_from_header(elements[row])
        if (name is not None and len(name)>0):
            columns.append(name)
    except:
        pass
columns

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'N/A',
 'Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 'FH 2',
 'FH 3',
 'Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 

## TASK 3: Create a data frame by parsing the launch HTML tables

In [11]:
launch_dict= dict.fromkeys(columns)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

In [12]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            launch_dict['Flight No.'].append(flight_number) 
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value 
            date = datatimelist[0].strip(',')
            launch_dict['Date'].append(date)
            #print(date)
            
            # Time value
            time = datatimelist[1]
            launch_dict['Time'].append(time)
            #print(time)
              
            # Booster version
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            launch_dict['Version Booster'].append(bv)
            #print(bv)
            
            # Launch Site
            launch_site = row[2].a.string
            launch_dict['Launch site'].append(launch_site)
            #print(launch_site)
            
            # Payload
            payload = row[3].a.string
            launch_dict['Payload'].append(payload)
            #print(payload)
            
            # Payload Mass
            payload_mass = get_mass(row[4])
            launch_dict['Payload mass'].append(payload_mass)
            #print(payload)
            
            # Orbit
            orbit = row[5].a.string
            launch_dict['Orbit'].append(orbit)
            # print(row[5].a)
            
            # Customer
            try:
                
                if row[6].a.string == None:
                    launch_dict['Customer'].append(None)
                else:
                    launch_dict['Customer'].append(row[6].a.string)
            except:
                pass
            # print(row[6].a)
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0]
            launch_dict['Launch outcome'].append(launch_outcome)
            #print(launch_outcome)
            
            # Booster landing
            booster_landing = landing_status(row[8])
            launch_dict['Booster landing'].append(booster_landing)
            #print(booster_landing)

In [13]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [14]:
df

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,N/A,FH 2,FH 3,...,People,Vehicles,Launches by rocket type,Launches by spaceport,"Agencies, companies and facilities",Other mission lists and timelines,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 B5B1051.10,Success,9 May 2021,06:42
117,118,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 B5B1058.8,Success,15 May 2021,22:56
118,119,CCSFS,Starlink,"15,600 kg",LEO,NASA,Success\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 B5B1063.2,Success,26 May 2021,18:59
119,120,KSC,SpaceX CRS-22,"3,328 kg",LEO,Sirius XM,Success\n,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,F9 B5B1067.1,Success,3 June 2021,17:29


In [15]:
df['Customer'].value_counts()

Customer
NASA                      32
SpaceX                    30
Iridium Communications     8
SES                        5
USAF                       3
ABS                        2
CONAE                      2
SKY Perfect JSAT Group     2
NRO                        2
AsiaSat                    2
Orbcomm                    2
Thaicom                    2
Sirius XM                  2
Telesat                    2
USSF                       1
Telkom Indonesia           1
Republic of Korea Army     1
U.S. Space Force           1
Sky Perfect JSAT           1
Spacecom                   1
Canadian Space Agency      1
PSN                        1
Spaceflight Industries     1
Es'hailSat                 1
KT Corporation             1
Thales-Alenia              1
Hispasat                   1
Hisdesat                   1
Northrop Grumman           1
SES S.A.                   1
NSPO                       1
Intelsat                   1
Bulsatcom                  1
Inmarsat                   1
EchoS